<a href="https://colab.research.google.com/github/BrenoMilagres/DataScienceProjetos/blob/master/predicaoNumeroGols.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo Preditivo para inferir um intervalo no qual está contido o número de gols de uma partida de futebol

O cenário de apostas esportivas no Brasil tem crescido exponencialmente nos últimos anos. Com o aumento de casas de apostas e o investimento feito para atrair cada vez mais jogadores, o setor tem sido um dos grandes destaques no país e movimenta cada vez mais dinheiro.

Existem diferentes tipos de apostas. Os tipos mais comuns de apostas são: simples, combinados e cruzadas.



- Simples
>Seu próprio nome o indica, elas são as mais simples apostas. São apostas em que jogamos o dinheiro em um único evento, é tão fácil quanto apostar em uma equipe ganhando ou perdendo. Se o resultado final coincidir com a previsão feita, venceremos a aposta e o lucro obtido será o resultado da multiplicação das chances pelo valor apostado.

- Combinadas
> Nas apostas combinadas, fazemos duas ou mais previsões na mesma aposta, sem que esses eventos necessariamente tenham que ser com o mesmo esporte. Para ganhar a aposta, será necessário que todas as previsões feitas coincidam com o resultado final.

- Cruzadas
> Nas apostas cruzadas, as casas de apostas agem como meros intermediários entre os usuários. Nesse tipo de aposta, são os apostadores que criam as probabilidades, os usuários que apostam a favor concordam com aqueles que o fazem cruzando suas apostas.

Modelos preditivos podem ser usados para prever a probabilidade do número total de gols marcados em uma partida de futebol. Isso poderia ser usado para obter uma vantagem nos mercados de apostas e obter lucro ao longo do tempo.


In [ ]:
#importando os dados para o google colab
from google.colab import files
uploaded = files.upload()

Saving database.sqlite to database (4).sqlite


In [ ]:
import numpy as np 
import pandas as pd 
import sqlite3 

with sqlite3.connect("database (4).sqlite") as con:
  match_data = pd.read_sql_query("SELECT * from Match", con)
match_data.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,...,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,None,None,None,None,None,None,None,None,1.73,3.40,5.00,1.75,3.35,4.20,1.85,3.2,3.5,1.80,3.3,3.75,NaN,NaN,NaN,1.70,3.30,4.33,1.90,3.3,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,None,None,None,None,None,None,None,None,1.95,3.20,3.60,1.80,3.30,3.95,1.90,3.2,3.5,1.90,3.2,3.50,NaN,NaN,NaN,1.83,3.30,3.60,1.95,3.3,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,None,None,None,None,None,None,None,None,2.38,3.30,2.75,2.40,3.30,2.55,2.60,3.1,2.3,2.50,3.2,2.50,NaN,NaN,NaN,2.50,3.25,2.40,2.63,3.3,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,None,None,None,None,None,None,None,None,1.44,3.75,7.50,1.40,4.00,6.80,1.40,3.9,6.0,1.44,3.6,6.50,NaN,NaN,NaN,1.44,3.75,6.00,1.44,4.0,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,None,None,None,None,None,None,None,None,5.00,3.50,1.65,5.00,3.50,1.60,4.00,3.3,1.7,4.00,3.4,1.72,NaN,NaN,NaN,4.20,3.40,1.70,4.50,3.5,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


#### Pré-processamento 
Será necessário pré-processar os dados para torná-los adequados para análise. Isso significa transformá-los de modo que cada partida tenha dois registros, um para cada equipe, com uma série de características que, espera-se, tenham algum mérito preditivo.

Além disso, para essa análise, serão consideradas apenas partidas da liga Inglesa mais conhecida como Premier league.

In [ ]:
match_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25979 entries, 0 to 25978
Columns: 115 entries, id to BSA
dtypes: float64(96), int64(9), object(10)
memory usage: 22.8+ MB


In [ ]:
# Selecionando partidas da Premier League.
match_data = match_data.loc[match_data['league_id'] == 1729]

# Selecionando colunas interessantes a análise.
columns = ['season',
           'stage',
           'match_api_id',
           'home_team_api_id',
           'away_team_api_id',
           'home_team_goal',
           'away_team_goal']

match_data = match_data[columns]
# Criando uma nova coluna com a diferença de gols entre os times mandante e visitante.
match_data['goal_difference'] = match_data['home_team_goal'] - match_data['away_team_goal']

match_data.head()

,season,stage,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,goal_difference
1728,2008/2009,1,489042,10260,10261,1,1,0
1729,2008/2009,1,489043,9825,8659,1,0,1
1730,2008/2009,1,489044,8472,8650,0,1,-1
1731,2008/2009,1,489045,8654,8528,2,1,1
1732,2008/2009,1,489046,10252,8456,4,2,2


In [ ]:
columns=['season', 
         'stage', 
         'match_id', 
         'team', 
         'opponent', 
         'team_goals', 
         'opponent_goals', 
         'goal_difference']
# Criando um registro para cada time 
md1 = match_data
md2 = match_data

md1.columns = columns
md2.columns = columns

md2 = md2.rename(columns={'opponent':'team','team':'opponent','team_goals':'opponent_goals','opponent_goals':'team_goals'})

md1['home_away'] = 1
md2['home_away'] = 0

md2['goal_difference'] = md2['goal_difference'] *-1

match_data = md1.append(md2)
match_data.sort_values(['season', 'team', 'stage'], inplace=True)
match_data.head()

,season,stage,match_id,team,opponent,team_goals,opponent_goals,goal_difference,home_away
1737,2008/2009,1,489051,8455,8462,4,0,4,1
1847,2008/2009,2,489061,8455,8528,1,0,1,0
1957,2008/2009,3,489071,8455,8586,1,1,0,1
2051,2008/2009,4,489075,8455,8456,3,1,2,0
2067,2008/2009,5,489091,8455,10260,1,1,0,1


Criação de 3 colunas arbitrárias:

- média de gols marcados nos últimos 3 jogos
- diferença da média de gols nos últimos 3 jogos
- média de gols sofridos nos últimos 3 jogos

In [ ]:
# média dos gols marcados útimos 3 jogos.
match_data['avg_team_goals'] = (match_data.groupby(['season','team'])['team_goals'].shift(1)+
match_data.groupby(['season','team'])['team_goals'].shift(2)+
match_data.groupby(['season','team'])['team_goals'].shift(3))/3

# média da diferença de gols ultimmos 3 jogos.
match_data['avg_goal_diff'] = (match_data.groupby(['season','team'])['goal_difference'].shift(1)+
match_data.groupby(['season','team'])['goal_difference'].shift(2)+
match_data.groupby(['season','team'])['goal_difference'].shift(3))/3

# média dos gols sofridos ultimos 3 jogos.
match_data['avg_goals_conc'] = (match_data.groupby(['season','team'])['opponent_goals'].shift(1)+
match_data.groupby(['season','team'])['opponent_goals'].shift(2)+
match_data.groupby(['season','team'])['opponent_goals'].shift(3))/3

match_data['avg_goals_conc'] = match_data['avg_goals_conc'] *-1

match_data = match_data.loc[match_data['avg_team_goals'].notnull()]



In [ ]:
match_data.head()

,season,stage,match_id,team,opponent,team_goals,opponent_goals,goal_difference,home_away,avg_team_goals,avg_goal_diff,avg_goals_conc
2051,2008/2009,4,489075,8455,8456,3,1,2,0,2.000000,1.666667,-0.333333
2067,2008/2009,5,489091,8455,10260,1,1,0,1,1.666667,1.000000,-0.666667
2076,2008/2009,6,489100,8455,10194,2,0,2,0,1.666667,0.666667,-1.000000
2087,2008/2009,7,489111,8455,10252,2,0,2,1,2.000000,1.333333,-0.666667
2092,2008/2009,8,489116,8455,8549,5,0,5,0,1.666667,1.333333,-0.333333



Descartando campos desnecessários e transformando o dado categórico de 'stage' em algo mais simples de ser processado.

Agora, para cada equipe e cada jogo, existe as 3 características explicadas acima, também se essa equipe está em casa ou fora - as equipes que jogam fora de casa  tendem a marcar menos - e em que fase da temporada o jogo está sendo disputado.

Em seguida, o conjunto de dados é dividido em variáveis ​​independentes e de destino(alvo da predição).

dividindo os resultados em acima ou abaixo de 3 gols como teste. Para ajustar o Regressor Logístico a esta variável de destino, é preciso transformar a coluna team_goals em 1 para mais de 3 gols ou 0 para menos de 3 gols.

In [ ]:
match_data = match_data[['match_id',
                         'home_away',
                         'stage',
                         'team_goals',
                         'avg_team_goals',
                         'avg_goal_diff',
                         'avg_goals_conc']]

match_data = match_data.reset_index(drop=True)

match_data['stage'] = pd.qcut(match_data['stage'], 5, labels=[1,2,3,4,5])

In [ ]:
md1 = match_data.loc[match_data['home_away'] == 1]
md2 = match_data.loc[match_data['home_away'] == 0]
md1.head()

,match_id,home_away,stage,team_goals,avg_team_goals,avg_goal_diff,avg_goals_conc
1,489091,1,1,1,1.666667,1.000000,-0.666667
3,489111,1,1,2,2.000000,1.333333,-0.666667
5,489131,1,1,0,3.000000,3.000000,-0.000000
7,489151,1,2,5,2.666667,2.333333,-0.333333
10,489181,1,2,0,3.333333,3.333333,-0.000000


In [ ]:
match_data = md1.merge(md2, on='match_id', how='left')
match_data['total_goals'] = match_data['team_goals_x'] + match_data['team_goals_y']
match_data = match_data.drop(['home_away_x', 'home_away_y', 'stage_y', 'match_id', 'team_goals_x', 'team_goals_y'], axis=1)


In [ ]:
match_data = match_data.rename(columns={'stage_x':'stage'})
match_data['stage'] = pd.qcut(match_data['stage'], 5, labels=[1,2,3,4,5])
match_data.head()

,stage,avg_team_goals_x,avg_goal_diff_x,avg_goals_conc_x,avg_team_goals_y,avg_goal_diff_y,avg_goals_conc_y,total_goals
0,1,1.666667,1.000000,-0.666667,1.666667,1.000000,-0.666667,2
1,1,2.000000,1.333333,-0.666667,2.000000,1.000000,-1.000000,2
2,1,3.000000,3.000000,-0.000000,2.666667,1.333333,-1.333333,1
3,2,2.666667,2.333333,-0.333333,0.666667,0.000000,-0.666667,5
4,2,3.333333,3.333333,-0.000000,1.666667,0.333333,-1.333333,0


In [ ]:
X = match_data[['stage', 'avg_team_goals_x', 'avg_goal_diff_x', 'avg_goals_conc_x',
                'avg_team_goals_y', 'avg_goal_diff_y', 'avg_goals_conc_y']]

match_data['faixa_degols'] = match_data['total_goals'].apply(lambda x: 0 if x<=2 else 1)

y = match_data['faixa_degols']
match_data.head()




,stage,avg_team_goals_x,avg_goal_diff_x,avg_goals_conc_x,avg_team_goals_y,avg_goal_diff_y,avg_goals_conc_y,total_goals,faixa_degols
0,1,1.666667,1.000000,-0.666667,1.666667,1.000000,-0.666667,2,0
1,1,2.000000,1.333333,-0.666667,2.000000,1.000000,-1.000000,2,0
2,1,3.000000,3.000000,-0.000000,2.666667,1.333333,-1.333333,1,0
3,2,2.666667,2.333333,-0.333333,0.666667,0.000000,-0.666667,5,1
4,2,3.333333,3.333333,-0.000000,1.666667,0.333333,-1.333333,0,0


In [ ]:

#importando o preprocessamento com o MinMax
from sklearn.preprocessing import MinMaxScaler
sc_X = MinMaxScaler()
X_escaler = sc_X.fit_transform(X)


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_escaler, y, test_size=0.2, random_state=0)



In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_pred = classifier.predict(X_test)
y_pred_p = classifier.predict_proba(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print("Acurácia para a classificação :  ",accuracy_score(y_pred,y_test))

Acurácia para a classificação :   0.5285714285714286


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[ 21, 248],
       [ 16, 275]])

In [ ]:
from sklearn.model_selection import cross_val_score
classifier = LogisticRegression(random_state = 0)
print(cross_val_score(classifier, X, y,cv=5))

[0.50535714 0.50357143 0.53392857 0.51428571 0.46964286]


In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train).feature_importances_

array([0.12311333, 0.1266718 , 0.1549355 , 0.13714289, 0.14866866,
       0.16502204, 0.14444579])

In [ ]:
classifier = RandomForestClassifier()
print(cross_val_score(classifier, X, y,cv=10))

[0.53214286 0.53571429 0.52142857 0.49285714 0.46785714 0.475
 0.49285714 0.45       0.52142857 0.52857143]


In [ ]:
len(X_test)

560

In [ ]:
from sklearn.svm import SVC
classifier = SVC()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 29, 240],
       [ 42, 249]])

In [ ]:
classifier.score(X,y)

0.5228571428571429

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[111, 158],
       [137, 154]])